# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [13]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [14]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [16]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [17]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [18]:
# Create a Keyspace 
session.execute("CREATE KEYSPACE IF NOT EXISTS udacity \
                WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")

#### Set Keyspace

In [19]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data
I write out the queries below to aid in designing the CREATE statements.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
> SELECT artist_name, song_title, song_length FROM table1 WHERE session_id = 338 AND itemInSession = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
> SELECT artist_name, song_title, first_name, last_name FROM table2 WHERE user_id = 10 AND session_id = 182

sorted by itemInSession means it needs to be part of the primary key as a clustering column.

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
> SELECT first_name, last_name FROM table3 WHERE song_title='All Hands Against His Own'



# Query description:

## Query 1:
Here the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key. Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.

## Query 2:
Here the primary key is composed of three items: user_id, session_id and itemInSession. To store the data together on a node, user_id and session_id are grouped together (with the ()) to create a composite partition key. The clustering column is itemInSession. Due to the way the data is stored in Cassandra, for a given user_id and session_id the itemInSession items are automatically sorted.

## Query 3:
Here the primary key is composed of two items: the song_title and user_id. The primary key needs to be unique so given the requirement for this query that can identify any user that listens to a particular song the combination of song_title and user_id should be unique.

In [20]:
session.execute("DROP TABLE IF EXISTS music_library")
session.execute("DROP TABLE IF EXISTS user_listening_library")
session.execute("DROP TABLE IF EXISTS who_listened_to")

# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
create_table1_query = "CREATE TABLE IF NOT EXISTS music_library (session_id int, itemInSession int, artist_name text, song_title text, song_length double, primary key(session_id, itemInSession))"
session.execute(create_table1_query)

# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
create_table2_query = "CREATE TABLE IF NOT EXISTS user_listening_library (user_id int, session_id int, itemInSession int, artist_name text, song_title text, first_name text, last_name text, primary key((user_id, session_id), itemInSession))"
session.execute(create_table2_query)

# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# I've added user_id to create a unique primary key
create_table3_query = "CREATE TABLE IF NOT EXISTS who_listened_to (song_title text, artist_name text, first_name text, last_name text, user_id int, primary key(song_title, user_id))"
session.execute(create_table3_query)

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query1 = "INSERT INTO music_library (session_id, itemInSession, artist_name, song_title, song_length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
        query2 = "INSERT INTO user_listening_library (user_id, session_id, itemInSession, artist_name, song_title, first_name, last_name)"
        query2 += "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
        query3 = "INSERT INTO who_listened_to (song_title, user_id, artist_name, first_name, last_name)"
        query3 += "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[0], line[1], line[4]))
        

#### Do a SELECT to verify that the data have been inserted into each table

# Query description

The purpose of query 1 is to find the artist, song and its length that was listened to for a given session and particular item in that session.

In [22]:
# Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("SELECT artist_name, song_title, song_length FROM music_library WHERE session_id = 338 AND itemInSession = 4")
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


# Query description
The purpose of query 2 is to find out what set of songs (and artists) a particular user with first & last name is listening to and in what order the songs have been listened to for a given user and session id.

In [23]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
rows = session.execute("SELECT artist_name, song_title, first_name, last_name, itemInSession FROM user_listening_library WHERE user_id = 10 AND session_id = 182")
for row in rows:
    print(row.artist_name, '-', row.song_title, '-', row.first_name, row.last_name, row.iteminsession)

                    

Down To The Bone - Keep On Keepin' On - Sylvie Cruz 0
Three Drives - Greece 2000 - Sylvie Cruz 1
Sebastien Tellier - Kilometer - Sylvie Cruz 2
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie Cruz 3


# Query description
The purpose of query 3 is to find all users that have listened to a particular song.

In [24]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
rows = session.execute("SELECT first_name, last_name FROM who_listened_to WHERE song_title='All Hands Against His Own'")
for row in rows:
    print(row.first_name, row.last_name)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Cleaning up: dropping tables

In [ ]:
# drop tables
session.execute("DROP TABLE IF EXISTS music_library")
session.execute("DROP TABLE IF EXISTS user_listening_library")
session.execute("DROP TABLE IF EXISTS who_listened_to")


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()